In [ ]:
!pip freeze | grep scikit-learn

In [ ]:
import pickle
import pandas as pd

In [ ]:
with open('model.bin', 'rb') as f_in:
    dv, model = pickle.load(f_in)

In [ ]:
categorical = ['PULocationID', 'DOLocationID']

def read_data(filename):
    df = pd.read_parquet(filename)
    
    df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
    df['duration'] = df.duration.dt.total_seconds() / 60

    df = df[(df.duration >= 1) & (df.duration <= 60)].copy()

    df[categorical] = df[categorical].fillna(-1).astype('int').astype('str')
    
    return df

In [ ]:
df = read_data('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2022-02.parquet')

In [ ]:
dicts = df[categorical].to_dict(orient='records')
X_val = dv.transform(dicts)
y_pred = model.predict(X_val)

In [ ]:
import statistics

In [ ]:
statistics.stdev(y_pred)

In [ ]:
year = 2022
month = 2
df['ride_id'] = f'{year:04d}/{month:02d}_' + df.index.astype('str')

In [ ]:
df.head(2)

In [ ]:
df_result = df[["ride_id"]].assign(prediction=y_pred)

In [ ]:
df_result.to_parquet(
    "prediction.parquet",
    engine='pyarrow',
    compression=None,
    index=False
)

In [ ]:
!ls -lthr

In [ ]:
!jupyter nbconvert --to python start.ipynb 